In [1]:
# pygame 설치
pip install pygame

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [14]:
import pygame 
import random  

pygame.init()
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 800
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT)) 
clock = pygame.time.Clock() 

# 이미지 위치 
location_url = 'asset/shoot_rock/'

#화면 색상 변수
screenColor_Black = (0, 0, 0)
screenColor_Red = (255, 0, 0)
screenColor_Green = (0, 255, 0)
screenColor_Blue = (0, 0, 255)
screenColor_White = (255, 255, 255)
screenColor_Gray = (128, 128, 128)
screenColor_Yellow = (255, 255, 0)

# 이미지 변수 생성 
background_image = pygame.image.load(location_url + 'background.png')
rock_image = pygame.image.load(location_url + 'rock.png')
fighter_image = pygame.image.load(location_url + 'fighter.png')
attack_image = pygame.image.load(location_url + 'missile.png')


# 변수 및 배열 선언
fighter = fighter_image.get_rect(centerx=SCREEN_WIDTH // 2, bottom=SCREEN_HEIGHT)
rocks = []
attacks = []

for i in range(3):
    rock = rock_image.get_rect(left=random.randint(0, SCREEN_WIDTH - rock_image.get_width()), top= -100)
    rocks.append(rock)

# pygame.key.set_repeat(1, 1)

while True: #게임 루프
    # 화면 지우기 - 검은색
    screen.fill(screenColor_Black)  
    screen.blit(background_image, (0, 0))

    #변수 업데이트

    event = pygame.event.poll() #이벤트 처리
    if event.type == pygame.QUIT:
        break
    elif event.type == pygame.KEYDOWN:
        if event.key == pygame.K_SPACE:
            attack = pygame.Rect(attack_image.get_rect(centerx=fighter.centerx, top=fighter.top))
            attacks.append(attack)
        
    pressed = pygame.key.get_pressed()
    if pressed[pygame.K_LEFT]:
        fighter.left -= 5
    elif pressed[pygame.K_RIGHT]:
        fighter.left += 5
    
    # 장애물 로직
    for rock in rocks:
        rock.top += 5
        if rock.top > SCREEN_HEIGHT:
            rocks.remove(rock)
            rock = rock_image.get_rect(left=random.randint(0, SCREEN_WIDTH - rock_image.get_width()), top= -100)
            rocks.append(rock)
            
    # 공격 로직
    for attack in attacks:
        attack.top -= 6
        if attack.top < 0:
            attacks.remove(attack)
    
    
    # 비행기 이동 로직 
    if fighter.left < 0:
        fighter.left = 0
    elif fighter.right > SCREEN_WIDTH:
        fighter.right = SCREEN_WIDTH
    
    # 장애물 격파 로직 
    for rock in rocks:
        for attack in attacks:
            if attack.colliderect(rock):
                print("rock attack!")
                rocks.remove(rock)
                attacks.remove(attack)
                rock = rock_image.get_rect(left=random.randint(0, SCREEN_WIDTH - rock_image.get_width()), top= -100)
                rocks.append(rock)
    
    #화면 그리기
    
    for rock in rocks:
        screen.blit(rock_image, rock)
    
    for attack in attacks:
        screen.blit(attack_image, attack)
    
    
    screen.blit(fighter_image, fighter)
    
    
    pygame.display.update() 
    # FPS 프레임
    clock.tick(30) 
    
    
pygame.quit() 

rock attack!
rock attack!
rock attack!
rock attack!
rock attack!
rock attack!
